### DATAFRAME

In [1]:
from cloudvolume import CloudVolume
from meshparty import skeletonize, trimesh_io
from caveclient import CAVEclient
import trimesh
import numpy as np
import datetime
import networkx as nx
from scipy.sparse import identity
from scipy.spatial import distance_matrix
import scipy 
from tqdm import tqdm
# import aws
import pandas as pd
import csv
import pyembree
import matplotlib.pyplot as plt
import scipy.spatial as spatial
import itertools
import math

In [2]:
orphans = pd.read_csv("/Users/sheeltanna/Desktop/AGT_REPO/campfire/ALL_ORPHANS.csv")

In [3]:
def my_array(x):
    res = list(map(str.strip, x.split('; ')))
    return res

In [4]:
orphans['endpoints'] = orphans['endpoints'].map(lambda x: list(map(str.strip, x.split('; '))))

In [5]:
## convert from string list to 2-d array
def convert_to_array(row):
    count = 0
    result = []
    for endpoint in row["endpoints"]:
        endpoint = eval(endpoint)
        if(count == 0):
            result = np.array(endpoint)
            count = count + 1
        else:
            result = np.vstack((result, np.array(endpoint)))
            count = count + 1
    if(count == 1 and result.size != 0):
        result = result.reshape(1,3)
    return result 

In [6]:
orphans["real_endpoints"] = orphans.apply(convert_to_array, axis = 1)

### TIP FINDER FUNCTIONS

In [7]:
def get_and_process_mesh(root_id):
    datastack_name = "minnie65_phase3_v1"
    client = CAVEclient(datastack_name)
    vol = CloudVolume(
        client.info.segmentation_source(),
        use_https=True,
        progress=False,
        bounded=False,
        fill_missing=True,
        secrets={"token": client.auth.token}
    )
    print("Downloading Mesh")
    mesh = vol.mesh.get(str(root_id))[root_id]
    mesh_obj = trimesh.Trimesh(np.divide(mesh.vertices, np.array([1,1,1])), mesh.faces)
    print("Vertices: ", mesh.vertices.shape[0])

    if mesh_obj.volume > 4000000000000:
        print("TOO BIG, SKIPPING")
        return None
    trimesh.repair.fix_normals(mesh_obj)
    mesh_obj.fill_holes()

    return mesh_obj

In [8]:
def get_soma(soma_id:str):
    cave_client = CAVEclient('minnie65_phase3_v1')
    soma = cave_client.materialize.query_table(
        "nucleus_neuron_svm",
        filter_equal_dict={'id':soma_id}
    )
    return soma

In [9]:
def process_mesh_ccs(mesh_obj):
    print("Processing CC's")
    ccs_graph = trimesh.graph.connected_components(mesh_obj.edges)
    ccs_len = [len(c) for c in ccs_graph]
    largest_component = ccs_graph[np.argmax(ccs_len)]
    largest_component_remap = np.arange(ccs_graph[np.argmax(ccs_len)].shape[0])
    face_dict = {largest_component[i]:largest_component_remap[i] for i in range(largest_component.shape[0])}

    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces = new_faces[new_faces[:, 0] != None]
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[largest_component], new_faces)

    all_ids = set(largest_component)
    encapsulated_ids = []

    for i in range(1, len(ccs_graph)):
        n_con = largest_component_mesh.contains(mesh_obj.vertices[ccs_graph[i]])
        if np.sum(n_con) / n_con.shape[0] == 0 and n_con.shape[0] > 50:
            all_ids.update(ccs_graph[i])
        else:
            if len(ccs_graph[i]) < 1000:
                encapsulated_ids.append((np.mean(mesh_obj.vertices[ccs_graph[i]], axis=0)/[4,4,40], len(ccs_graph[i])))
            
    all_component = np.array(list(ccs_graph[np.argmax(ccs_len)]))
    all_component_remap = np.arange(all_component.shape[0])
    face_dict = {all_component[i]:all_component_remap[i] for i in range(all_component.shape[0])}
    new_faces_mask = np.isin(mesh_obj.faces, list(face_dict.keys()))
    new_faces_mask = new_faces_mask[:, 0]*new_faces_mask[:, 1]*new_faces_mask[:, 2]

    new_faces = np.vectorize(face_dict.get)(mesh_obj.faces[new_faces_mask])
    new_faces[new_faces[:, 0] != None]
    
    largest_component_mesh = trimesh.Trimesh(mesh_obj.vertices[all_component], new_faces)
    
    mesh_obj = largest_component_mesh
    return mesh_obj, encapsulated_ids, np.max(ccs_len)

In [10]:
def process_defects(mesh_obj, a=.75):
    bad_edges = trimesh.grouping.group_rows(
        mesh_obj.edges_sorted, require_count=1)
    bad_edges_ind = mesh_obj.edges[bad_edges]
    sparse_edges = mesh_obj.edges_sparse
    xs = list(bad_edges_ind[:, 0]) + list(bad_edges_ind[:, 1]) 
    ys = list(bad_edges_ind[:, 1]) + list(bad_edges_ind[:, 0])
    vs = [1]*bad_edges_ind.shape[0]*2
    bad_inds = scipy.sparse.coo_matrix((vs, (xs, ys)), shape=(mesh_obj.vertices.shape[0], mesh_obj.vertices.shape[0]))
    # Make it symmetrical and add identity so each integrates from itself too, then subtract singleton edges
    # I noticed that the number of asymmetrical edges vs the number of single edges I find from group rows
    # Are close but different. Haven't looked into that yet. Also removing edges 1 hop away from single edges to remove bias towards
    # Holes in the mesh that are caused by mesh construction errors as opposed to segmentation errors
    sparse_edges = mesh_obj.edges_sparse + mesh_obj.edges_sparse.T + identity(mesh_obj.edges_sparse.shape[0]) - sparse_edges.multiply(bad_inds) - bad_inds
    degs = mesh_obj.vertex_degree + 1

    # N_iter is a smoothing parameter here. The loop below smooths the vertex error about the mesh to get more consistent connected regions
    n_iter = 2
    angle_sum = np.array(abs(mesh_obj.face_angles_sparse).sum(axis=1)).flatten()
    defs = (2 * np.pi) - angle_sum

    abs_defs = np.abs(defs)
    abs_defs_i = abs_defs.copy()
    for i in range(n_iter):
        abs_defs_i = sparse_edges.dot(abs_defs_i) / degs
    
    verts_select = np.argwhere((abs_defs_i > a))# & (abs_defs < 2.5))

    edges_mask = np.isin(mesh_obj.edges, verts_select)
    edges_mask[bad_edges] = False
    edges_select = edges_mask[:, 0] * edges_mask[:, 1]
    edges_select = mesh_obj.edges[edges_select]

    G = nx.from_edgelist(edges_select)#f_edge_sub)

    ccs = nx.connected_components(G)
    subgraphs = [G.subgraph(cc).copy() for cc in ccs]

    lens = []
    lengths = []
    for i in tqdm(range(len(subgraphs))):
        ns = np.array(list(subgraphs[i].nodes()))
    #     ns = ns[abs_defs[ns ]]
        l = len(ns)
        if l > 20 and l < 5000:
            lens.append(ns)
            lengths.append(l)
    all_nodes = set()
    for l in lens:
        all_nodes.update(l)
    all_nodes = np.array(list(all_nodes))
    # sharp_pts = mesh_obj.vertices[all_nodes]
    centers = np.array([np.mean(mesh_obj.vertices[list(ppts)],axis=0) for ppts in lens])

    return centers, lens

In [11]:
def process_endpoints(mesh_obj, skel_mp):
    # Process the skeleton to get the endpoints
    interior_cc_mask = set()
    el = nx.from_edgelist(skel_mp.edges)
    comps = list(nx.connected_components(el))
    for c in comps:
        if len(c) < 100:
            n_con = mesh_obj.contains(skel_mp.vertices[list(c)])
            if np.sum(n_con) / n_con.shape[0] > .10:
                interior_cc_mask.update(list(c))
    # Process the skeleton to get the endpoints
    edges = skel_mp.edges.copy()

    edge_mask = ~np.isin(edges, interior_cc_mask)
    edge_mask = edge_mask[:, 0] + edge_mask[:, 1]
    edges = edges[edge_mask]
    edges_flat  = edges.flatten()
    edge_bins = np.bincount(edges_flat) 

    eps = np.squeeze(np.argwhere(edge_bins==1))
    eps_nm = skel_mp.vertices[eps]

    eps_comp = distance_matrix(eps_nm, eps_nm)
    eps_comp[eps_comp == 0] = np.inf
    eps_thresh = np.argwhere(~(np.min(eps_comp, axis=0) < 3000))

    eps = np.squeeze(eps[eps_thresh])
    eps_nm = np.squeeze(eps_nm[eps_thresh])
    return eps, eps_nm

In [12]:
def process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp):
    if eps.size == 1:
        #eps_nm should therefore be reshaped
        eps_nm = eps_nm.reshape((1,3))

    print("Processing mesh errors")
    # path_to_root_dict = {}
    # for ep in eps:
    #     path_to_root_dict[ep] = skel_mp.path_to_root(ep)
        
    dists_defects = np.zeros(centers.shape[0])
    sizes = np.zeros(centers.shape[0])
    mesh_map = skel_mp.mesh_to_skel_map
    closest_skel_pts = mesh_map[[l[0] for l in lens]]

    # print(centers, eps_nm)

    dist_matrix = distance_matrix(centers, eps_nm)
    ct = 0

    closest_tip = np.zeros((centers.shape[0]))

    for center in tqdm(centers):
    #     skel_pts_dists = np.linalg.norm(skel_mp.vertices - center, axis=1)
    #     ep_pts_dists = np.linalg.norm(eps_nm - center, axis=1)
        
        # closest_skel_pt = closest_skel_pts[ct]
        min_ep = np.inf
        eps_hit = []
        if eps.size == 1:
            eps_hit.append(eps)
        else: 
            for j, ep in enumerate(eps):
                # if closest_skel_pt in path_to_root_dict[ep]:
                #     eps_hit.append(j)
                eps_hit.append(j)
        
        if(eps.size == 1):
            dists = dist_matrix[ct]
        else:
            dists = dist_matrix[ct, eps_hit]
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip[ct] = tip_hit
    #     print(np.argmin(ep_pts_dists), ep_found, eps_nm[np.argmin(ep_pts_dists)]/[4,4,40], eps_nm[j]/[4,4,40], center/[4,4,40])
        dists_defects[ct] = min_dist
        sizes[ct] = len(lens[ct])
        ct+=1
    dists_defects_sub = dists_defects[dists_defects < 5000]
    sizes_sub = sizes[dists_defects < 5000]
    centers_sub = centers[dists_defects < 5000]
    tips_hit_sub = closest_tip[dists_defects < 5000]
    closest_skel_pts_sub = closest_skel_pts[dists_defects < 5000]
    inds_sub = np.arange(centers.shape[0])[dists_defects < 5000]


    # Also ranking each component based on its PCA- if the first component is big enough, the points are mostly linear
    # These point sets seem to be less likely to be true errors
    from sklearn.decomposition import PCA
    pca_vec = np.zeros(inds_sub.shape[0])
    for i in range(inds_sub.shape[0]):
        pca = PCA()#n_components=2)
        pca.fit(mesh_obj.vertices[lens[inds_sub[i]]])

        pca_vec[i] = pca.explained_variance_ratio_[0]

    dists_defects_sub[dists_defects_sub < 4000] = 100
    dists_defects_norm = dists_defects_sub #/ np.max(dists_defects_sub)
    ranks_ep = sizes_sub / dists_defects_norm * (1-pca_vec)
    ranks = sizes_sub**2 * (1-pca_vec)

    #ranks_ep_errors_filt = ranks_ep[ranks_ep > .1]
    centers_ep_send_errors = centers_sub[np.argsort(ranks_ep)][::-1][:20]
    final_mask_eps = np.full(centers_ep_send_errors.shape[0], True)
    tips_hit_send_ep = tips_hit_sub[np.argsort(ranks_ep)][::-1][:20]
    uns, nums = np.unique(tips_hit_send_ep, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_eps[np.argwhere(tips_hit_send_ep == un)[1:]] = False
    centers_errors_ep = centers_ep_send_errors[final_mask_eps]
    centers_errors = centers_sub[np.argsort(ranks)[::-1]][:20]
    return centers_errors, centers_errors_ep

In [13]:
def process_mesh_facets(mesh_obj, skel_mp, eps, eps_nm, facet_area_threshold=30000):

    
    if eps.size == 1:
        #eps_nm should therefore be reshaped
        eps_nm = eps_nm.reshape((1,3))

    #changed threshold of facets to 3000 to match changes performed on processed in analysis CELLS
    print("Processing facets")
    #can possibly change param here
    #threshold on size of flat area
    locs = np.argwhere(mesh_obj.facets_area > facet_area_threshold)

    mesh_map = skel_mp.mesh_to_skel_map
    mesh_coords = mesh_obj.vertices[mesh_obj.faces]
    mean_locs = []
    mesh_ind = []
    fs = []
    for l in tqdm(locs):
        fs.append(np.sum(mesh_obj.facets_area[l]))
        fc = mesh_obj.facets[l[0]]
        vert_locs = mesh_coords[fc]
        mean_locs.append(np.mean(vert_locs[:, 0], axis=0))
        mesh_ind.append(fc[0])
    mesh_ind = mesh_obj.faces[mesh_ind][:, 0]
    mean_locs = np.array(mean_locs)
    if mean_locs.size == 0:
        facets_send_final = np.zeros((1,3))
        return facets_send_final
    dists_defects_facets = np.zeros(mean_locs.shape[0])
    mesh_map_facets = skel_mp.mesh_to_skel_map
    closest_skel_pts_facets = mesh_map[[m for m in mesh_ind]]
    dist_matrix_facets = distance_matrix(mean_locs, eps_nm)
    ct = 0

    closest_tip_facets = np.zeros((mean_locs.shape[0]))

    for center in tqdm(mean_locs):

        closest_skel_pt = closest_skel_pts_facets[ct]
        eps_hit = []
        ##added for error fixing
        if eps.size == 1:
            eps_hit.append(eps)
        else:
            for j, ep in enumerate(eps):
                # if closest_skel_pt in path_to_root_dict[ep]:
                #     eps_hit.append(j)
                eps_hit.append(j)
        
            #distance from every error(facets and jagged) to every endpoint
        if(eps.size == 1):
            dists = dist_matrix_facets[ct]
        else:
            dists = dist_matrix_facets[ct, eps_hit]
    
        
        amin = np.argmin(dists)
        tip_hit = eps_hit[amin]
        min_dist = dists[amin]
        
        closest_tip_facets[ct] = tip_hit
        dists_defects_facets[ct] = min_dist
        ct+=1
    dists_defects_sub_facets = dists_defects_facets[dists_defects_facets < 2000]
    sizes_sub_facets = np.array(fs)[dists_defects_facets < 2000]
    mean_locs_facets = mean_locs[dists_defects_facets < 2000]
    tips_hit_sub_facets = closest_tip_facets[dists_defects_facets < 2000]
    closest_skel_pts_sub_facets = closest_skel_pts_facets[dists_defects_facets < 2000]
    inds_sub_facets = np.arange(mean_locs.shape[0])[dists_defects_facets < 2000]
    # ranks_ep_facets = sizes_sub_facets**2 / dists_defects_sub_facets
    #ranks_ep_facets_filt = ranks_ep_facets[ranks_ep_facets > 2e7]
    mean_locs_send_facets = mean_locs_facets
    final_mask_facets = np.full(mean_locs_send_facets.shape[0], True)
    tips_hit_send_facets = tips_hit_sub_facets
    uns, nums = np.unique(tips_hit_send_facets, return_counts=True)

    for un, num in zip(uns, nums):
        if num > 1:
            final_mask_facets[np.argwhere(tips_hit_send_facets == un)[1:]] = False
    facets_send_final = mean_locs_send_facets[final_mask_facets] / [4,4,40]
    return facets_send_final

### TIP FINDER FUNCTION

In [14]:
def error_locs_defects(root_id, soma_id = None, soma_table=None, center_collapse=True):
    mesh_obj = get_and_process_mesh(root_id)
    if mesh_obj is None:
        return None
    try:
        if soma_table==None:
            soma_table = get_soma(str(soma_id))
        if soma_table[soma_table.id == soma_id].shape[0] > 0:
            center = np.array(soma_table[soma_table.id == soma_id].pt_position)[0] * [4,4,40]
        else:
            center=None
    except:
        center = None
    print("Subselecting largest connected component of mesh")
    mesh_obj, encapsulated_ids, max_verts = process_mesh_ccs(mesh_obj)
    

    skel_mp = skeletonize.skeletonize_mesh(trimesh_io.Mesh(mesh_obj.vertices, 
                                            mesh_obj.faces),
                                            invalidation_d=4000,
                                            shape_function='cone',
                                            collapse_function='branch',
#                                             soma_radius = soma_radius,
                                            soma_pt=center,
                                            smooth_neighborhood=5,
                                             cc_vertex_thresh=max_verts - 10
#                                                     collapse_params = {'dynamic_threshold':True}
                                            )
    print("Skel done")
    centers, lens = process_defects(mesh_obj)
    eps, eps_nm = process_endpoints(mesh_obj, skel_mp)

    if(len(eps) == 0):
        sorted_encapsulated_send = np.zeros((1,3))
        facets_send_final = np.zeros((1,3))
        errors_send = np.zeros((1,3))
        errors_tips_send = np.zeros((1,3))
        return sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send
        
    
    if len(centers) !=0:
        centers_errors, centers_errors_ep = process_mesh_errors(mesh_obj, centers, eps, eps_nm, lens, skel_mp)
    else:
        centers_errors = np.zeros ((1,3))
        centers_errors_ep = np.zeros ((1,3))
        ranks = np.zeros ((1))
        ranks_ep = np.zeros((1, 3))
        ranks_return = 0
        ranks_ep_return = 0
    facets_send_final = process_mesh_facets(mesh_obj, skel_mp, eps, eps_nm)
    errors_send = centers_errors / [4,4,40]
    errors_tips_send = centers_errors_ep / [4,4,40]
    encapsulated_centers = [e[0] for e in encapsulated_ids]
    encapsulated_lens = [e[1] for e in encapsulated_ids]
    sorted_encapsulated_send = np.array(encapsulated_centers)[np.argsort(encapsulated_lens)][::-1]
    return sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send

### Generating Endpoints

In [15]:
def find_endpoints(row):
    seg_id = row["seg_id"]
    print(seg_id)
    sorted_encapsulated_send, facets_send_final, errors_send, errors_tips_send = error_locs_defects(seg_id)
    together = np.vstack((facets_send_final, errors_tips_send))
    mask=np.sum(together,axis=1)
    together = together[mask > 0]
    return together

In [16]:
def generate_endpoints(dataframe) :
    dataframe["endpoints_generated"] = dataframe.apply(find_endpoints, axis = 1)
    return dataframe

### Accuracy Function

In [17]:
def pred_eps_acc(gt_endpoints, pred_endpoints, threshold):
    # Calculate distances
    dist_matrix = np.array(spatial.distance.cdist(gt_endpoints, pred_endpoints, metric = 'euclidean'))
    # Apply threshold
    dist_matrix[dist_matrix > threshold] = 0
    # Calculating accuracy
    valid_eps = np.count_nonzero(dist_matrix, axis = 1)
    accuracy = np.count_nonzero(valid_eps) / len(gt_endpoints)
    return accuracy

### Testing On Orphans

In [18]:
generate_endpoints(orphans)

864691135909994000
Vertices:  7993
Subselecting largest connected component of mesh
Processing CC's


100%|██████████| 7992/7992 [00:00<00:00, 1097354.16it/s]


Skel done


100%|██████████| 26/26 [00:00<00:00, 165732.38it/s]


Processing facets


...

Processing mesh errors


100%|██████████| 13/13 [00:00<00:00, 78908.76it/s]

Processing facets



100%|██████████| 66/66 [00:00<00:00, 59404.31it/s]


,neuron,ng_link,seg_id,pink_pts,num_endpoints,endpoints,comments,detailed_comments,real_endpoints,endpoints_generated
0,864691136577830164,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"[(402584, 228856, 23991), (402985, 229235, 235...",good,NaN,"[[402584, 228856, 23991], [402985, 229235, 235...","[[402970.1091549296, 229214.63028169013, 23552..."
1,864691136577830164,https://neuroglancer.neuvue.io/?json_url=https...,864691135247440303,"(401258, 224832, 24029)",3,"[(401612, 224623, 23991), (405257, 226318, 236...",good,NaN,"[[401612, 224623, 23991], [405257, 226318, 236...","[[405014.6845794392, 226607.61448598132, 23618..."
2,864691136577830164,https://neuroglancer.neuvue.io/?json_url=https...,864691134794123793,"(401314, 228366, 24424)",2,"[(401242, 228382, 24444), (400982, 228457, 245...",m,merged to two axon pieces,"[[401242, 228382, 24444], [400982, 228457, 245...",[]
3,864691136577830164,https://neuroglancer.neuvue.io/?json_url=https...,864691135772363453,"(400199, 220721, 24029)",7,"[(401289, 218721, 23991), (399895, 216533, 235...",m,NaN,"[[401289, 218721, 23991], [399895, 216533, 235...","[[399920.325, 216558.3, 23517.375], [401217.25..."
4,864691136577830164,https://neuroglancer.neuvue.io/?json_url=https...,864691135314714227,"(397870, 232292, 24004)",2,"[(397867, 232230, 23999), (397854, 232252, 239...",good,NaN,"[[397867, 232230, 23999], [397854, 232252, 239...",[]
...,...,...,...,...,...,...,...,...,...,...
119,864691135526309723,https://neuroglancer.neuvue.io/?json_url=https...,864691135089638001,"(122738, 154726, 20654)",2,"[(122789, 154701, 20649), (122688, 151314, 205...",good,NaN,"[[122789, 154701, 20649], [122688, 151314, 205...","[[122716.125, 154749.0, 20649.3]]"
120,864691135526309723,https://neuroglancer.neuvue.io/?json_url=https...,864691135319600870,"(106639, 111664, 20802)",2,"[(106608, 111803, 20796), (98290, 97555, 21045)]",good,NaN,"[[106608, 111803, 20796], [98290, 97555, 21045]]","[[106823.5, 111737.5, 20733.3], [98335.2794117..."
121,864691135526309723,https://neuroglancer.neuvue.io/?json_url=https...,864691136558839249,"(103176, 188510, 21202)",2,"[(103138, 188530, 21200), (91780, 189141, 20879)]",good,NaN,"[[103138, 188530, 21200], [91780, 189141, 20879]]","[[91735.62096774194, 189119.98790322582, 20877..."
122,864691135526309723,https://neuroglancer.neuvue.io/?json_url=https...,864691135458639120,"(127406, 206585, 21529)",2,"[(127475, 206611, 21525), (128120, 207267, 215...",good,NaN,"[[127475, 206611, 21525], [128120, 207267, 215...","[[127722.0, 206896.375, 21513.45], [127879.5, ..."


In [19]:
#apply function to entire df 
count = 0
acc_array = []
for index, row in orphans.iterrows():
    if (type(row["real_endpoints"])== list and type(row["endpoints_generated"]) == list):
        acc = 1
        print("both empty")
    elif(type(row["endpoints_generated"]) == list and type(row["real_endpoints"]) != list):
        acc = 0
        print("no endpoints generated, but endpoints exist")
    else:
        count = count + 1
        acc = pred_eps_acc(row["real_endpoints"], row["endpoints_generated"], 250)
        acc_array.append(acc)
        if(acc == 0.0):
            print(row["seg_id"])

864691134794123793
864691135314714227
864691135552841025
864691134330810585
864691133692934758
864691133097965890
864691133946526057
864691133712131152
864691135559405139
864691134472630080
864691133678427777
864691133922173572
864691134742288112
864691136668362099
864691133766052845
864691134282775283
864691135496889954
864691135154890214
864691135619456283
864691133769133345
864691135066358591
864691133676046827
864691133661730792
864691132747855184
864691135196787278
864691136347887475
864691134890549721
864691133525367772
864691134016784879
864691135461715680
864691134182969537
864691136584897745
864691132647441812
864691136597387729
864691135458639120


In [20]:
acc_array = np.array(acc_array)
print(acc_array)

[1.         1.         0.         0.75       0.         0.
 1.         0.4        0.         0.4        0.5        1.
 0.5        0.         0.         1.         0.         0.
 0.         1.         0.         0.         0.         1.
 0.         0.         0.33333333 0.         0.5        0.5
 0.         0.5        0.         0.5        0.5        0.66666667
 0.5        0.6        1.         0.85714286 0.66666667 0.66666667
 0.5        0.33333333 1.         0.5        0.45454545 0.5
 0.66666667 0.66666667 0.33333333 0.5        0.5        0.
 0.         0.25       0.66666667 0.66666667 1.         1.
 1.         1.         0.5        0.         0.5        1.
 0.66666667 1.         0.85714286 0.5        0.5        0.8
 0.         0.5        1.         0.5        0.5        1.
 1.         0.5        0.66666667 0.5        0.5        0.5
 0.         0.         0.66666667 0.5        0.5        0.5
 0.5        1.         0.5        0.         1.         0.5
 1.         0.         1.         

In [27]:
counter = 0
for acc in acc_array:
    if acc >= 0.3:
        counter = counter + 1

print(counter)

88


In [23]:
len(acc_array)

124